In [1]:
!pip install scikit-fuzzy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894079 sha256=50e8848eb04906799139976d7dd05673c4e3dfd06d2d25dfcd3d8bb7693d3cbc
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [2]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt


In [3]:
# Entradas
febre = ctrl.Antecedent(np.arange(36, 41.1, 0.1), 'febre')
dor_cabeca = ctrl.Antecedent(np.arange(0, 10.1, 1), 'dor_cabeca')  # 0: Nenhuma, 10: Máxima
tosse = ctrl.Antecedent(np.arange(0, 10.1, 1), 'tosse')  # 0: Nenhuma, 10: Persistente

# Saída
diagnostico = ctrl.Consequent(np.arange(0, 8.1, 1), 'diagnostico')  # 8 possíveis diagnósticos

# Definindo as funções de pertinência para a febre
febre['baixa'] = fuzz.trimf(febre.universe, [36, 36.5, 37.5])
febre['media'] = fuzz.trimf(febre.universe, [37, 37.5, 38.5])
febre['alta'] = fuzz.trimf(febre.universe, [38, 38.5, 41])

# Definindo as funções de pertinência para a dor de cabeça
dor_cabeca['leve'] = fuzz.trimf(dor_cabeca.universe, [0, 2, 4])
dor_cabeca['moderada'] = fuzz.trimf(dor_cabeca.universe, [3, 5, 7])
dor_cabeca['intensa'] = fuzz.trimf(dor_cabeca.universe, [6, 8, 10])

# Definindo as funções de pertinência para a tosse
tosse['ocasional'] = fuzz.trimf(tosse.universe, [0, 2, 4])
tosse['frequente'] = fuzz.trimf(tosse.universe, [3, 5, 7])
tosse['persistente'] = fuzz.trimf(tosse.universe, [6, 8, 10])




In [12]:
# Definindo os conjuntos fuzzy para os diagnósticos
diagnostico['desconhecido'] = fuzz.trimf(diagnostico.universe, [15, 17, 19])

diagnostico['infeccoes_virais_leves'] = fuzz.trimf(diagnostico.universe, [0, 1, 2])
diagnostico['resfriado_comum'] = fuzz.trimf(diagnostico.universe, [1, 2, 3])
diagnostico['gripe'] = fuzz.trimf(diagnostico.universe, [2, 3, 4])
diagnostico['sinusite'] = fuzz.trimf(diagnostico.universe, [3, 4, 5])
diagnostico['bronquite'] = fuzz.trimf(diagnostico.universe, [4, 5, 6])
diagnostico['pneumonia'] = fuzz.trimf(diagnostico.universe, [5, 6, 7])
diagnostico['infeccao_bacteriana'] = fuzz.trimf(diagnostico.universe, [6, 7, 8])
diagnostico['alergias_respiratorias'] = fuzz.trimf(diagnostico.universe, [7, 8, 9])


In [23]:


# Regras para Infecções Virais Leves
regra1 = ctrl.Rule(febre['baixa'] & dor_cabeca['leve'] & tosse['ocasional'], diagnostico['infeccoes_virais_leves'])
regra2 = ctrl.Rule(febre['baixa'] & dor_cabeca['leve'] & tosse['frequente'], diagnostico['infeccoes_virais_leves'])

# Regras para Resfriado Comum
regra3 = ctrl.Rule(febre['baixa'] & dor_cabeca['moderada'] & tosse['ocasional'], diagnostico['resfriado_comum'])
regra4 = ctrl.Rule(febre['baixa'] & dor_cabeca['leve'] & tosse['persistente'], diagnostico['resfriado_comum'])

# Regras para Gripe
regra5 = ctrl.Rule(febre['media'] & dor_cabeca['moderada'] & tosse['frequente'], diagnostico['gripe'])
regra6 = ctrl.Rule(febre['media'] & dor_cabeca['intensa'] & tosse['ocasional'], diagnostico['gripe'])

# Regras para Sinusite
regra7 = ctrl.Rule(febre['baixa'] & dor_cabeca['intensa'] & tosse['ocasional'], diagnostico['sinusite'])

# Regras para Bronquite
regra8 = ctrl.Rule(febre['media'] & dor_cabeca['leve'] & tosse['persistente'], diagnostico['bronquite'])
regra9 = ctrl.Rule(febre['alta'] & dor_cabeca['leve'] & tosse['persistente'], diagnostico['bronquite'])

# Regras para Pneumonia
regra10 = ctrl.Rule(febre['alta'] & dor_cabeca['moderada'] & tosse['persistente'], diagnostico['pneumonia'])
regra11 = ctrl.Rule(febre['alta'] & dor_cabeca['intensa'] & tosse['persistente'], diagnostico['pneumonia'])

# Regras para Infecção Bacteriana
regra12 = ctrl.Rule(febre['alta'] & dor_cabeca['intensa'] & tosse['frequente'], diagnostico['infeccao_bacteriana'])

# Regras para Alergias Respiratórias
regra13 = ctrl.Rule(febre['baixa'] & dor_cabeca['moderada'] & tosse['frequente'], diagnostico['alergias_respiratorias'])
regra14 = ctrl.Rule(febre['media'] & dor_cabeca['leve'] & tosse['ocasional'], diagnostico['alergias_respiratorias'])



# Agrupando todas as regras
regras = [regra1, regra2, regra3, regra4, regra5, regra6, regra7, regra8, regra9, regra10, regra11, regra12, regra13, regra14,]






In [18]:
# Criando o sistema de controle fuzzy
sistema_diagnostico = ctrl.ControlSystem(regras)

# Criando a simulação do sistema de controle
simulacao_diagnostico = ctrl.ControlSystemSimulation(sistema_diagnostico)


In [15]:
!pip install ipywidgets

In [27]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np  # Importando a biblioteca numpy para verificar NaN

# Criando os widgets para entrada de dados
input_febre = widgets.FloatText(value=37.0, description='Febre (°C):', step=0.1)
input_dor_cabeca = widgets.IntSlider(value=5, min=0, max=10, description='Dor de Cabeça:')
input_tosse = widgets.IntSlider(value=5, min=0, max=10, description='Tosse:')

# Botão para realizar a simulação
btn_simular = widgets.Button(description="Simular Diagnóstico")

# Função para converter o valor float de diagnóstico em texto descritivo
def interpretar_diagnostico(valor):
    if np.isnan(valor):  # Verificando se o valor é NaN
        return "Diagnóstico Desconhecido"
    elif valor <= 2.5:
        return "Infecções Virais Leves"
    elif 2.5 < valor <= 4.5:
        return "Resfriado Comum"
    elif 4.5 < valor <= 6.5:
        return "Gripe"
    elif 6.5 < valor <= 8.5:
        return "Sinusite"
    elif 8.5 < valor <= 10.5:
        return "Bronquite"
    elif 10.5 < valor <= 12.5:
        return "Pneumonia"
    elif 12.5 < valor <= 14.5:
        return "Infecção Bacteriana"
    elif 14.5 < valor <= 16.5:
        return "Alergias Respiratórias"
    else:
        return "Diagnóstico não conclusivo"

# Função para realizar a simulação quando o botão for pressionado
def on_simulate_clicked(b):
    # Definindo as entradas para a simulação
    simulacao_diagnostico.input['febre'] = input_febre.value
    simulacao_diagnostico.input['dor_cabeca'] = input_dor_cabeca.value
    simulacao_diagnostico.input['tosse'] = input_tosse.value

    try:
        # Computando o diagnóstico
        simulacao_diagnostico.compute()
        # Interpretando o resultado
        diagnostico_texto = interpretar_diagnostico(simulacao_diagnostico.output['diagnostico'])
    except ValueError:
        diagnostico_texto = "Diagnóstico Desconhecido"

    # Exibindo o resultado
    clear_output(wait=True)
    display(input_febre, input_dor_cabeca, input_tosse, btn_simular)
    print(f"Diagnóstico Sugerido: {diagnostico_texto}")

# Associando a função ao botão
btn_simular.on_click(on_simulate_clicked)

# Exibindo os widgets
display(input_febre, input_dor_cabeca, input_tosse, btn_simular)


FloatText(value=40.2, description='Febre (°C):', step=0.1)

IntSlider(value=8, description='Dor de Cabeça:', max=10)

IntSlider(value=2, description='Tosse:', max=10)

Button(description='Simular Diagnóstico', style=ButtonStyle())

Diagnóstico Sugerido: Diagnóstico Desconhecido


# Instruções:


***Febre*** (°C):

36°C a 37.5°C: Normal ou ligeiramente elevada.

37°C a 38.5°C: Febre moderada.

38°C a 41°C: Febre alta.

***Dor de Cabeça*** (0-10):


0 a 4: Leve.

5 a 7: Moderada.

8 a 10: Intensa.


***Tosse*** (0-10)

0 a 4: Ocasional.

5 a 7: Frequente.

8 a 10: Persistente.
